# Getting Team Data

In [1]:
import json
import time
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
# Base URL
base_url = 'https://www.basketball-reference.com'

### Team Name and URLs

In [3]:
# Appending the '/teams/' to Base URL
url = base_url + '/teams/'

# Sending get request to the page
response = requests.get(url)

# Creating BeautifulSoup object for scrapping
soup = BeautifulSoup(response.content, 'lxml')

# Getting all active team name from the BeautifulSoup object
team_data = soup.find(id='all_teams_active').tbody.find_all('th', {'data-stat': 'franch_name'})

# Empty dictionary for storing team URLs and team names
team_urls = {}

# Appending the team's URL and team_name
for data in team_data:
    team_urls[data.text] = base_url+data.a['href']
    
# Writing team URLs into a file
with open('.\\data\\team_urls.json', 'w', encoding='utf8') as file_obj:
    json.dump(team_urls, file_obj)

### Season URLs

In [4]:
# Dictionary to store team URLs for each season
season_urls = {}

pbar = tqdm(total=len(team_urls), desc='Team Name')

for team_name in team_urls:
    pbar.update(1)
    pbar.set_description(team_name)
    
    # Sending get request to each team data
    response = requests.get(team_urls.get(team_name))
    
    #Creating a BeautifulSoup object for scrapping
    soup = BeautifulSoup(response.content, 'lxml')
    
    # Extracting each season href for all years
    season_data = soup.tbody.find_all('td', {"data-stat": "team_name"})
    
    # Empty list for storing season URLs
    team_seasons = []
    
    # Adding season URLs from 2023 to 2010 season
    for season_href in season_data[:14]:
        team_seasons.append(base_url+season_href.a['href'])
    
    # Adding season URLs with the team name to the dictionary
    season_urls[team_name] = team_seasons
    
    # Sleeping due to rate limitations
    time.sleep(2)

# Writing the season URLs data into a file
with open('.\\data\\season_urls.json', 'w', encoding='utf8') as file_obj:
    json.dump(season_urls, file_obj)

Washington Wizards: 100%|██████████████████████████████████████████████████████████████| 30/30 [01:10<00:00,  2.44s/it]